<center><h1>Singh_Avantika_HW3</h1></center>
<br>
<br>

Name: Avantika Singh
<br>
Github Username: avantikasingh
<br>
USC ID: 6318290290

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import warnings
from io import StringIO
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
from tabulate import tabulate
warnings.filterwarnings('ignore')

### (b) Test and Train Data

In [2]:
root="../data/AReM"
train = []
test = []
for root, dir, file in os.walk(root):
    for f in file:
        if 'pdf' not in f and '.DS_Store' not in f:
            fpof = os.path.join(root, f)
            if 'bending1' in fpof or 'bending2' in fpof:
                if 'dataset1.csv' in fpof or 'dataset2.csv' in fpof:
                    test.append(fpof)
                else:
                    train.append(fpof)
            else:
                if 'dataset1.csv' in fpof or 'dataset2.csv' in fpof or 'dataset3.csv' in fpof:
                    test.append(fpof)
                else:
                    train.append(fpof)


In [3]:
train

['../data/AReM/bending1/dataset7.csv',
 '../data/AReM/bending1/dataset6.csv',
 '../data/AReM/bending1/dataset4.csv',
 '../data/AReM/bending1/dataset5.csv',
 '../data/AReM/bending1/dataset3.csv',
 '../data/AReM/walking/dataset7.csv',
 '../data/AReM/walking/dataset6.csv',
 '../data/AReM/walking/dataset4.csv',
 '../data/AReM/walking/dataset5.csv',
 '../data/AReM/walking/dataset10.csv',
 '../data/AReM/walking/dataset11.csv',
 '../data/AReM/walking/dataset13.csv',
 '../data/AReM/walking/dataset12.csv',
 '../data/AReM/walking/dataset15.csv',
 '../data/AReM/walking/dataset14.csv',
 '../data/AReM/walking/dataset8.csv',
 '../data/AReM/walking/dataset9.csv',
 '../data/AReM/bending2/dataset6.csv',
 '../data/AReM/bending2/dataset4.csv',
 '../data/AReM/bending2/dataset5.csv',
 '../data/AReM/bending2/dataset3.csv',
 '../data/AReM/standing/dataset7.csv',
 '../data/AReM/standing/dataset6.csv',
 '../data/AReM/standing/dataset4.csv',
 '../data/AReM/standing/dataset5.csv',
 '../data/AReM/standing/dataset

In [4]:
test

['../data/AReM/bending1/dataset1.csv',
 '../data/AReM/bending1/dataset2.csv',
 '../data/AReM/walking/dataset1.csv',
 '../data/AReM/walking/dataset2.csv',
 '../data/AReM/walking/dataset3.csv',
 '../data/AReM/bending2/dataset1.csv',
 '../data/AReM/bending2/dataset2.csv',
 '../data/AReM/standing/dataset1.csv',
 '../data/AReM/standing/dataset2.csv',
 '../data/AReM/standing/dataset3.csv',
 '../data/AReM/sitting/dataset1.csv',
 '../data/AReM/sitting/dataset2.csv',
 '../data/AReM/sitting/dataset3.csv',
 '../data/AReM/lying/dataset1.csv',
 '../data/AReM/lying/dataset2.csv',
 '../data/AReM/lying/dataset3.csv',
 '../data/AReM/cycling/dataset1.csv',
 '../data/AReM/cycling/dataset2.csv',
 '../data/AReM/cycling/dataset3.csv']

### (c) Feature Extraction

#### i. Research

Time-domain features used in time series classification:



> **Mean (Average):** Represents the central tendency of the data

> **Median:** Middle value in a sorted list of data points and less sensitive to outliers compared to the mean

> **Minimum:** Smallest value in the time series

> **Maximum:** Largest value in the time series

> **Standard Deviation:** Measures the spread or dispersion of the data points from the mean

> **Variance:** Quantifies how much the data points vary from the mean

> **Skewness:** Measures the asymmetry of the data distribution

> **Kurtosis:** Measures the "tailedness" or the heaviness of the tails in the data distribution

> **Range:** Difference between the maximum and minimum values

> **Quantiles:** Quantiles divide the data into equal intervals, with the first quartile (25th percentile) and the third quartile (75th percentile) being commonly used.

> **Entropy:** Quantifies the amount of information or disorder in the data

>  **Autocorrelation:** Measures the correlation between a time series and a lagged version of itself.

> **Cross-Correlation:** Measures the correlation between two different time series

> **Energy:** Represents the total energy contained in the time series

> **Zero Crossing Rate:** Counts the number of times the signal changes sign


#### ii. Extraction

In [28]:
COLUMN_NAMES = [ "min1", "max1", "mean1", "median1", "std1", "1st quartile1", "3rd quartile1",
                 "min2", "max2", "mean2", "median2", "std2", "1st quartile2", "3rd quartile2",
                 "min3", "max3", "mean3", "median3", "std3", "1st quartile3", "3rd quartile3",
                 "min4", "max4", "mean4", "median4", "std4", "1st quartile4", "3rd quartile4",
                 "min5", "max5", "mean5", "median5", "std5", "1st quartile5", "3rd quartile5",
                 "min6", "max6", "mean6", "median6", "std6", "1st quartile6", "3rd quartile6"]

def get_delimiter(file):
    sniffer = csv.Sniffer()
    f = open(file)
    return sniffer.sniff(f.read()).delimiter


# Define function to load data from a folder
def load_data_from_folder(folder_path):
    data = pd.DataFrame()
    #files = os.listdir(folder_path)
    #print(folder_path)

    try:
      delimiter = get_delimiter(folder_path)
      #print(delimiter)
      df = []
      if(delimiter == ','):
        df = pd.read_csv(folder_path, delimiter=',', header=None, skiprows=5)
      elif(delimiter==' '):
        df = pd.read_csv(folder_path, delimiter='\\s+', header=None, skiprows=5)
      else:
        df = pd.read_csv(folder_path, header=None, skiprows=5, on_bad_lines='skip')

      df.drop(columns=df.columns[0], axis=1, inplace=True) #drop the 'time' column
      #print(df)
      data = pd.concat([data, df], axis=1)


    except Exception as e:
            # some files have one trailing comma at the end

            # Read the entire CSV file as text
            with open(folder_path, 'r') as input_file:
                csv_content = input_file.read()

            # Check if the CSV content ends with a comma
            if csv_content.endswith(','):
                # Remove the trailing comma from the end of the content
                csv_content = csv_content[:-1]  # Remove the last character (comma)

            # Use StringIO to read the modified CSV content as a DataFrame
            df = pd.read_csv(StringIO(csv_content),delimiter=',', header=None,skiprows=5)

            df.drop(columns=df.columns[0], axis=1, inplace=True) #drop the 'time' column
            data = pd.concat([data, df], axis=1)

    #print(data)
    return data


def extract_time_series_features(data):
  time_series_features = pd.DataFrame()
  for folder in data:
      folder_path = f'{folder}'
      df = load_data_from_folder(folder_path)
      # Calculate the describe statistics for each column
      describe_stats = df.describe().loc[['min', 'max', 'mean', '50%', 'std', '25%', '75%']].T
      #print(describe_stats)

      # Flatten the describe statistics into a single row
      flatten_stats = describe_stats.values.flatten()

      final_df = pd.DataFrame([flatten_stats], columns=COLUMN_NAMES)

      #extract folder name from file path
      second_last_index = folder.rfind("/", 0, folder.rfind("/"))
      final_df['folder'] = folder[second_last_index+1:folder.rfind("/")]

      time_series_features = pd.concat([time_series_features,final_df])
  
  time_series_features = time_series_features.reset_index(drop=True)
  return time_series_features

##### Time-domain Features for Training Data

In [31]:
train_features = pd.DataFrame()
train_features = extract_time_series_features(train)
display(train_features)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,1st quartile5,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300,bending1
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700,bending1
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000,bending1
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000,bending1
4,35.00,47.40,43.954500,44.33,1.558835,43.0000,45.00,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.9400,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600,cycling
65,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000,cycling
66,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175,cycling
67,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300,cycling


##### Time-domain Features for Test Data

In [32]:
test_features = extract_time_series_features(test)
display(test_features)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,1st quartile5,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing


##### New Dataset with all 88 instances

In [34]:
train_features['label'] = 0
test_features['label'] = 1
time_series_dataset = pd.concat([train_features, test_features])
display(time_series_dataset)

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder,label
0,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,23.7500,0.0,2.96,0.555312,0.49,0.487826,0.0000,0.83,bending1,0
1,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,24.0000,0.0,5.26,0.679646,0.50,0.622534,0.4300,0.87,bending1,0
2,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,36.3300,0.0,2.18,0.613521,0.50,0.524317,0.0000,1.00,bending1,0
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,31.2500,0.0,1.79,0.383292,0.43,0.389164,0.0000,0.50,bending1,0
4,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,36.5000,0.0,1.79,0.493292,0.43,0.513506,0.0000,0.94,bending1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,24.75,48.33,44.182937,48.000,7.495615,48.00,48.0000,0.0,3.11,0.101875,...,5.5425,0.0,3.91,0.692771,0.50,0.675781,0.3225,0.94,lying,1
15,48.00,48.25,48.004167,48.000,0.032038,48.00,48.0000,0.0,0.43,0.007167,...,10.0000,0.0,2.50,0.641229,0.50,0.388372,0.4600,0.83,lying,1
16,24.25,45.00,37.177042,36.250,3.581301,34.50,40.2500,0.0,8.58,2.374208,...,21.7500,0.0,9.34,2.921729,2.50,1.852600,1.5000,3.90,cycling,1
17,28.75,44.75,37.561187,36.875,3.226507,35.25,40.2500,0.0,9.91,2.080687,...,21.5000,0.0,9.62,2.765896,2.45,1.769203,1.4100,3.77,cycling,1


,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6,folder,label
0,36.25,48.00,43.969125,44.50,1.618364,43.31,44.67,0.0,1.50,0.413125,...,23.75,0.0,2.96,0.555312,0.49,0.487826,0.00,0.83,bending1,0
1,37.00,48.00,43.454958,43.25,1.386098,42.50,45.00,0.0,1.58,0.378083,...,24.00,0.0,5.26,0.679646,0.50,0.622534,0.43,0.87,bending1,0
2,33.00,47.75,42.179812,43.50,3.670666,39.15,45.00,0.0,3.00,0.696042,...,36.33,0.0,2.18,0.613521,0.50,0.524317,0.00,1.00,bending1,0
3,33.00,45.75,41.678063,41.75,2.243490,41.33,42.75,0.0,2.83,0.535979,...,31.25,0.0,1.79,0.383292,0.43,0.389164,0.00,0.50,bending1,0
4,35.00,47.40,43.954500,44.33,1.558835,43.00,45.00,0.0,1.70,0.426250,...,36.50,0.0,1.79,0.493292,0.43,0.513506,0.00,0.94,bending1,0


#### iii. Standard Deviation

In [31]:
def calculate_std_ci(dataset):
  std_dev_and_ci = {}

  for feature in dataset.columns:
      data = dataset[feature].values
      # Calculate the standard deviation of the original data
      std_dev = np.std(data)

      # Perform bootstrap resampling to create confidence intervals
      resamples = bs.bootstrap(data, stat_func=bs_stats.std, num_iterations=10000, num_threads=1)

      # Extract the 90% confidence interval
      lower_bound = resamples.lower_bound
      upper_bound = resamples.upper_bound

      std_dev_and_ci[feature] = {
          'Standard Deviation': std_dev,
          '90% CI Lower Bound': lower_bound,
          '90% CI Upper Bound': upper_bound
      }

  table = [['Feature', 'Standard Deviation','90% CI Lower Bound','90% CI Upper Bound']]
  for feature, values in std_dev_and_ci.items():
    table.append([feature,values['Standard Deviation'], values['90% CI Lower Bound'], values['90% CI Upper Bound']])


  print(tabulate(table, headers='firstrow', tablefmt='grid'))

In [32]:
calculate_std_ci(time_series_dataset)

+---------------+----------------------+----------------------+----------------------+
| Feature       |   Standard Deviation |   90% CI Lower Bound |   90% CI Upper Bound |
+===============+======================+======================+======================+
| min1          |             9.51545  |             8.0683   |           11.0037    |
+---------------+----------------------+----------------------+----------------------+
| max1          |             4.36932  |             3.28063  |            5.62627   |
+---------------+----------------------+----------------------+----------------------+
| mean1         |             5.30531  |             4.6396   |            6.04634   |
+---------------+----------------------+----------------------+----------------------+
| median1       |             5.40906  |             4.74857  |            6.14441   |
+---------------+----------------------+----------------------+----------------------+
| std1          |             1.76206  |   

#### iv. Select Features

Based on my understanding, for a regression problem I would pick the following three features to capture essential characteristics of time-series data:

*   **Mean**: It gives information about the central tendency of data which would be useful to interpret the overall trend of data

*   **Standard Deviation**: Measures the spread of data points around mean. Hence, this would be useful in identifying noise or anamolies in the time-series data

*   **Third Quartile**: Quartiles are robust summary statistics and are less sensitive to outliers compared to the maximum or minimum values. Third quartile indicats the value below which 75% of the data points fall.



## 2. ISLR 3.7.4

### (a) Linear Train

In this case, the cubic model might do better than the linear model since flexibility of the model is directly proportional on the number of parameters. Hence the cubic RSS will fit the model more closely than the linear model. There is not enough information, but atleast on the train data the RSS for cubic regression will be better or atleast same as the RSS for linear regression.

### (b) Linear Test

On Test Data, the linear regression will have lower RSS than cubic regression when the true relationship is linear. This is because the cubic regression has flexibility to capture nonlinear patterns that don't exist in the true relationship. This can lead to overfitting, where the model fits the noise in the data rather than the true underlying linear trend.

### (c) Not Linear Train

Cubic regression RSS would be lower than linear regression RSS since cubic model would be more flexible and would better fit the non-linear relationships.

### (d) Not Linear Testing

On Test data, it depends on the underlying relationship whether the linear regression would do better or the cubic regression. Hence, there is not enough information to a concrete answer. However, if the true relationship between X and Y is not very far from linear, the linear regression RSS would be lower. If the true relationship is very far from linear, the cubic regression RSS would be lower.

## 3. ISLR 3.7.3 - Extra Practice

## 3. ISLR 3.7.5 - Extra Practice